In [1]:
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup

In [2]:
my_url = 'https://www.rogue.com/beers'

#Opening up connection, grabbing the page
uClient = uReq(my_url)
page_html = uClient.read()
uClient.close()


In [3]:
#Parsing the html file, because it's a lot of text right now, into a variable
page_soup = soup(page_html, "html.parser")

#Test print, prints the header
page_soup.h1
page_soup.p


<p>Rogue brews a variety of ales and lagers at their World Headquarters in Newport, Oregon. Brewmaster Joel Shields is brewing up innovative beers and overseeing production on our 100 barrel brewing system. We bottle and can our ales and distribute them in all 50 states and 54 countries. Use the <a href="http://finder.rogue.com">Rogue Finder</a> to hunt down Rogue ales and lagers near you.</p>

In [4]:
#Grabs each product
containers = page_soup.findAll("li", {"class": "column product-grid-item is-half-mobile is-one-third-tablet is-one-third-widescreen is-one-quarter-fullhd"})

#How many objects did it find?
len(containers)


32

In [5]:
#Looking at the info for the first container
#container = containers[0]
#container

In [6]:
#Testing to pull out the link
#container.a["href"]

In [7]:
#Creating .csv file, instantiate a mode (w = write)
filename = "rogue_links.csv"
f = open(filename, "w")

headers = "Link to beers\n"

#Writing a header for the column
f.write(headers)

14

In [8]:
for container in containers:
    link = container.a["href"]
    f.write(link + "\n")

f.close()